In [ ]:
import os
from lumaai import LumaAI

# read .env file
from dotenv import load_dotenv
load_dotenv()


client = LumaAI(
    auth_token=os.environ["LUMA_TOKEN"],
)

import numpy as np
from PIL import Image, ImageDraw, ImageFilter


def soft_edge(size):

  width, height = size
  mask = Image.new("L", (width, height), 0)
  draw = ImageDraw.Draw(mask)
  edge_width = min(width, height) // 10

  for i in range(edge_width):
      draw.rectangle([i, i, width - i - 1, height - i - 1], fill=i * (255 // edge_width))

  blurred_mask = mask.filter(ImageFilter.GaussianBlur(radius=edge_width / 2))
  transparency_array = np.array(blurred_mask)

  return transparency_array/transparency_array.max()



generation = client.generations.image.create(
  prompt="image of forest",
)
completed = False
while not completed:
  generation = client.generations.get(id=generation.id)
  if generation.state == "completed":
    completed = True
  elif generation.state == "failed":
    raise RuntimeError(f"Generation failed: {generation.failure_reason}")
  print("Dreaming")
  time.sleep(2)

image_url = generation.assets.image

# download image_url into PIL
from PIL import Image
import requests
from io import BytesIO

response = requests.get(image_url)
img = Image.open(BytesIO(response.content))
background = img

import cv2
video_path = "smoke.mp4"
import random
video = cv2.VideoCapture(video_path)
masks = []
frames = []
blended = []


slience_frames = 10 * 30
for i in range(slience_frames):
  masks.append(np.zeros((512, 512), dtype=np.uint8))
  blended.append(background)


while video.isOpened():
  background = img.resize((512, 512)).convert("L")
  ret, frame = video.read()
  size = random.randint(128, 256)
  locx = random.randint(0, 512 - size[0])
  locy = random.randint(0, 512 - size[1])
  if not ret:
    break
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  gray = cv2.resize(gray, (512, 512))

  gray = Image.fromarray(gray).resize((size, size))
  
  white = Image.new('RGB', (512, 512), (255, 255, 255))

  canvas = Image.new('L', (512, 512), (0))
  softedge_mask = Image.fromarray((soft_edge((size, size)) * 255).astype("uint8"))
  # softedge_mask = Image.new('L', (512, 512), (0))
  # softedge_mask.paste(softedge_mask, (100, 100))
  canvas.paste(gray, (locx, locy), mask=softedge_mask)
  gray = np.array(canvas)
  mask = gray > 0.1 * 255
  gray = Image.fromarray(gray.astype("uint8"))

  masks.append(mask)
  frames.append(gray)
  background.paste(white, (100, 100), mask=Image.fromarray((np.array(gray)//2 * mask).astype("uint8")))
  blended.append(background)

# save blended into video
os.mkdir("in")
os.mkdir("gt")
for i in range(len(blended)):
  blended[i].save(f"in/{i}.png")
  masks[i].save(f"gt/{i}.png")


